In [6]:
import requests
import pandas as pd

In [7]:
# load .env and fetch api key
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')

In [8]:
def get_puuid(summonerId:str=None, gameName:str=None, tagLine:str=None, region:str='europe'):
    '''Given summonerId or gameName and tagLine of Riot Account returns the puuid of that account

    Arguments:
        str: summonerId, defualts to None.
        str: gameName, defualts to None.
        str: tagLine, defualts to None.
        str: region, defualts to 'europe'.

    Returns:
        str: puuid
    '''

    if summonerId is not None:
        # region tag is a silly little goose
        if region == 'europe':
            region = 'euw1'
        root = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        request = root + endpoint + summonerId + '?api_key=' + api_key
        response = requests.get(request)
    else:
        if gameName is None or tagLine is None:
            raise Exception("Please input gameName and tagLine")

        request = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'
        response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))
    puuid = response.json()['puuid']

    return puuid

In [9]:
# testing get_puuid
gameName1 = 'SONDER'
tagLine1 = 'mafk1'
summonerId1 = '0qDuJrMmD3ht2azWwkEZ1iAAuZ3wpWfcrKsTNHe9yIYoEfY'
print("puuid for SONDER#mafk1: " + get_puuid(summonerId=summonerId1))
print("puuid equal using summonerId vs gameName and tagLine?", get_puuid(gameName=gameName1, tagLine=tagLine1) == get_puuid(summonerId=summonerId1))

puuid for SONDER#mafk1: smKXivxW6VCPxTu9NyeWVNNObhayvp7njxu8EmGlOxQ94ttoGDGYp7a9-cgB68kxXUj2EhIWysGfVg
puuid equal using summonerId vs gameName and tagLine? True


In [10]:
def get_ranked_match_history(puuid:str=None, region:str='europe', n_matches:int=20):
    '''Given puuid of Riot Account, returns a list of match IDs as strings

    Arguments:
        str: puuid, defualts to None.
        str: region, defualts to 'europe'.
        int: n_matches, defualts to 20 and must be between 0 and 100 (inclusive).

    Returns:
        list: match IDs as strings
    '''
    
    if puuid is None:
        raise Exception('Please provide a puuid in string format.')
    if n_matches > 100:
        raise Exception('Amount of matches cannot exceed 100.')
    if n_matches < 0:
        raise Exception('Amount of matches must be at least 0.')

    root = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}'
    request = root + endpoint + '/ids?type=ranked&start=0&n_matches=' + str(n_matches) + '&api_key=' + api_key
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    return response.json()
    


In [11]:
# testing get_ranked_match_history
print("Amount of matches is 20?", len(get_ranked_match_history(puuid='smKXivxW6VCPxTu9NyeWVNNObhayvp7njxu8EmGlOxQ94ttoGDGYp7a9-cgB68kxXUj2EhIWysGfVg')) == 20)
get_ranked_match_history(puuid=get_puuid(summonerId=summonerId1))

Amount of matches is 20? True


['EUW1_6952527686',
 'EUW1_6952505511',
 'EUW1_6952481621',
 'EUW1_6952464749',
 'EUW1_6952440607',
 'EUW1_6952040165',
 'EUW1_6952003281',
 'EUW1_6951966254',
 'EUW1_6951935148',
 'EUW1_6951916614',
 'EUW1_6949831247',
 'EUW1_6949811541',
 'EUW1_6949786554',
 'EUW1_6949744355',
 'EUW1_6948333566',
 'EUW1_6948254395',
 'EUW1_6948237653',
 'EUW1_6947107179',
 'EUW1_6947079585',
 'EUW1_6947055813']

In [12]:
def get_match_data(matchId:str=None, region='europe'):
    '''Given League of Legends Match ID, returns a pandas dataframe of useful match data

    Arguments:
        str: matchId, defualts to None.
        str: region, defualts to 'europe'.

    Returns:
        dataframe: match data
    '''

    root = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'
    request = root + endpoint + '?api_key=' + api_key
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    df = pd.DataFrame.from_dict(response.json())

    return df
    

# What info does get_match_data dataframe contain?

## InfoDto

- gameVersion - contains patch info
- participants - list of ParticipantDto

## ParticipantDto

- championName -  name of champion played
- championTransform - used for Kayn transformation, which can either be 0 (None), 1 (Slayer) or 2 (Assassin)
- item0
- item1
- item2
- item3
- item4
- item5
- item6
- lane - lane played
- perks - perksDto

## PerksDto

- statPerks -  PerkStatsDto
- styles - list of PerkStyleDto

## PerkStatsDto

- defense
- flex
- offense

## PerkStyleDto

- description
- selections - list of PerkStyleSelectionDto
- style

## PerkStyleSelectionDto

- perk
- var1
- var2
- var3

## TeamDto

- win

In [15]:
champion = get_match_data(matchId='EUW1_6906634163')['info']['participants'][0]['championId']
win = get_match_data(matchId='EUW1_6906634163')['info']['participants'][0]['win']

print(champion)
print(win)

875
True


In [17]:
test_data = {
    "match_id": [],
    "champion": [],
    "win": []
}
test_df = pd.DataFrame(test_data)

print(test_df)

Empty DataFrame
Columns: [match_id, champion, win]
Index: []


In [40]:
import time

c = {
    "match_id": [],
    "champion": [],
    "win": []
}

gathered_data = pd.DataFrame(c)

print(gathered_data)

def gather_los_dataos(matches):
    count = 0
    for match in matches:
        if not match in gathered_data['match_id'].value_counts().keys():
            if count % 100 == 0 and count > 0: # make sure we don't exceed max number of API calls per 2 minutes
                time.sleep(120)
            info = get_match_data(matchId=match)['info']
            participants = info['participants']
            count += 1
            if not info['queueId'] == 420:
                continue
            for participant in participants:
                champion = participant['championId']
                win = participant['win']
                gathered_data.loc[len(gathered_data.index)] = [match, champion, win]

                
start_time = time.time()
#gather_los_dataos(['EUW1_6906634163', 'EUW1_6906634163'])
gather_los_dataos(get_ranked_match_history(puuid=get_puuid(gameName='Dunkvern', tagLine='EUW')))
print('Time elapsed: %s' % (time.time() - start_time))
print(gathered_data)

Empty DataFrame
Columns: [match_id, champion, win]
Index: []
420
420
1700
1700
1700
420
420
420
420
420
420
420
420
440
440
420
420
420
420
420
Time elapsed: 4.524739027023315
            match_id  champion    win
0    EUW1_6956655366       126  False
1    EUW1_6956655366       234  False
2    EUW1_6956655366        84  False
3    EUW1_6956655366       202  False
4    EUW1_6956655366        37  False
..               ...       ...    ...
145  EUW1_6952472199       266  False
146  EUW1_6952472199        79  False
147  EUW1_6952472199       517  False
148  EUW1_6952472199        51  False
149  EUW1_6952472199       902  False

[150 rows x 3 columns]


In [41]:
print(gathered_data['match_id'].value_counts())

match_id
EUW1_6956655366    10
EUW1_6956647517    10
EUW1_6956213134    10
EUW1_6956168957    10
EUW1_6955324041    10
EUW1_6955253299    10
EUW1_6954597565    10
EUW1_6954571990    10
EUW1_6954563868    10
EUW1_6954320659    10
EUW1_6952789569    10
EUW1_6952739969    10
EUW1_6952681798    10
EUW1_6952517112    10
EUW1_6952472199    10
Name: count, dtype: int64
